In [ ]:
if True:
    %rm -rf source
    !git clone https://github.com/HaiDang2001VN/albert-imdb.git source
    %pip install -U wandb transformers evaluate huggingface_hub accelerate
    colab = True
else:
    print("Not running on Google Colab, skip this cell!")
    colab = False

Cloning into 'source'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (224/224), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 224 (delta 116), reused 217 (delta 111), pack-reused 0
Receiving objects: 100% (224/224), 2.52 MiB | 8.36 MiB/s, done.
Resolving deltas: 100% (116/116), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 43.4 MB/s eta 0:00:00
     ━━━━━━━

In [ ]:
import json
from pprint import pprint

# Load path to pre-processed data in configs folder
config_file = "vlsp_colab" if colab else "vlsp_local"
with open(f"configs/{config_file}.json") as f:
    configs = json.load(f)

pprint(configs)

In [ ]:
import wandb

run_model = ["phobert", "base", "format0"]
wandb.init(project="advanced_ai_imdb_dataset", name=f"{run_model[0]}-{run_model[1]}-run-vlsp-{run_model[2]}")

In [ ]:
from datasets import load_dataset

source_path = configs["path"]
format_name = run_model[2]
# format_name = "format1"
# train_set = load_dataset(
#     "json", data_files=f"./vlsp_preprocessed/{format_name}/train.jsonl", split="train"
# )
# val_set = load_dataset(
#     "json", data_files=f"./vlsp_preprocessed/{format_name}/dev.jsonl", split="train"
# )
# test_set = load_dataset(
#     "json", data_files=f"./vlsp_preprocessed/{format_name}/test.jsonl", split="train"
# )
dataset = load_dataset(
    "json",
    data_files={
        "train": f"{source_path}/vlsp_preprocessed/{format_name}/train.jsonl",
        "val": f"{source_path}/vlsp_preprocessed/{format_name}/dev.jsonl",
        "test": f"{source_path}/vlsp_preprocessed/{format_name}/test.jsonl",
    },
)

In [ ]:
model_path = configs['model_path'][run_model[0]][run_model[1]]
print(model_path)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path,
                                                           num_labels=4,
                                                           output_hidden_states=False
                                                           )

In [ ]:
def preprocess_token(example):
    return tokenizer(
        example["sentence"], padding="max_length", truncation=True, return_tensors="pt"
    )

In [ ]:
tokenized_dataset = dataset.map(preprocess_token, batched=True)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")
precision = evaluate.load("precision")
recall = evaluate.load("recall")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    print(f"Type of {type(predictions)=}")
    print(f"Type of {type(labels)=}")
    print(f"Shape of {predictions[0].shape}")
    print(f"Shape of {predictions[1].shape}")
    if isinstance(predictions, tuple):
        predictions = predictions[0]
    predictions = np.argmax(predictions, axis=1)
    # predictions = np.argmax(predictions.reshape(-1, predictions.shape[-1]), axis=1)
    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels)['accuracy'],
        "f1_macro": f1.compute(predictions=predictions, references=labels, average="macro")['f1'],
        "f1_micro": f1.compute(predictions=predictions, references=labels, average="micro")['f1'],
        "f1_weighted": f1.compute(predictions=predictions, references=labels, average="weighted")['f1'],
        "precision_macro": precision.compute(predictions=predictions, references=labels, average="macro")['precision'],
        "precision_micro": precision.compute(predictions=predictions, references=labels, average="micro")['precision'],
        "precision_weighted": precision.compute(predictions=predictions, references=labels, average="weighted")['precision'],
        "recall_macro": recall.compute(predictions=predictions, references=labels, average="macro")['recall'],
        "recall_micro": recall.compute(predictions=predictions, references=labels, average="micro")['recall'],
        "recall_weighted": recall.compute(predictions=predictions, references=labels, average="weighted")['recall'],
    }

In [ ]:
id2label = {0: 'AFFILIATION', 1: 'PART – WHOLE', 2: 'LOCATED', 3: 'PERSONAL - SOCIAL'}
label2id = {value: key for key, value in id2label.items()}
print(label2id)

{'AFFILIATION': 0, 'PART – WHOLE': 1, 'LOCATED': 2, 'PERSONAL - SOCIAL': 3}


In [ ]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="albert_imdb",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=25,
    weight_decay=0.01,
    save_steps=0.1,
    load_best_model_at_end=True,
    overwrite_output_dir=True,
    save_total_limit=1,
    evaluation_strategy="steps",
    eval_steps=0.1,
    logging_steps=0.05,
    report_to="wandb",
    gradient_accumulation_steps=5,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
tokenized_dataset["train"][0]

{'sentence': 'Trong ảnh : Nghệ_thuật tạo hoa_văn trên trang_phục truyền_thống của người Mông hoa , tại <location> xã Sa_Lông </location> , <location> huyện Mường_Chà </location> .',
 'label': 2,
 'input_ids': [2,
  13,
  38,
  14271,
  40,
  252,
  13,
  45,
  13,
  2723,
  438,
  1,
  38,
  7325,
  38,
  14341,
  20538,
  1,
  2686,
  26407,
  13,
  38,
  13119,
  1,
  3971,
  6335,
  8600,
  8944,
  1,
  96,
  3279,
  13,
  22936,
  13,
  2723,
  13324,
  49,
  21028,
  20538,
  13,
  15,
  5466,
  13,
  1,
  19032,
  1,
  13,
  6791,
  1929,
  1,
  2701,
  13,
  1,
  118,
  19032,
  1,
  13,
  15,
  13,
  1,
  19032,
  1,
  4429,
  8944,
  2832,
  3279,
  1,
  1651,
  13,
  1,
  118,
  19032,
  1,
  13,
  9,
  3,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Micro,F1 Weighted,Precision Macro,Precision Micro,Precision Weighted,Recall Macro,Recall Micro,Recall Weighted
63,1.097500,0.475202,0.841402,0.866019,0.841402,0.843592,0.871212,0.841402,0.851553,0.865719,0.841402,0.841402
126,0.383900,0.432957,0.814691,0.818625,0.814691,0.812231,0.860568,0.814691,0.825216,0.791536,0.814691,0.814691
189,0.281800,0.308675,0.879800,0.896656,0.879800,0.879280,0.902773,0.879800,0.881063,0.892207,0.879800,0.879800
252,0.200400,0.277068,0.898164,0.912240,0.898164,0.897111,0.921658,0.898164,0.899484,0.905633,0.898164,0.898164
315,0.135200,0.308870,0.894825,0.910360,0.894825,0.894403,0.917921,0.894825,0.897033,0.905019,0.894825,0.894825


Type of type(predictions)=<class 'numpy.ndarray'>
Type of type(labels)=<class 'numpy.ndarray'>
Shape of (4,)
Shape of (4,)
Type of type(predictions)=<class 'numpy.ndarray'>
Type of type(labels)=<class 'numpy.ndarray'>
Shape of (4,)
Shape of (4,)
Type of type(predictions)=<class 'numpy.ndarray'>
Type of type(labels)=<class 'numpy.ndarray'>
Shape of (4,)
Shape of (4,)
Type of type(predictions)=<class 'numpy.ndarray'>
Type of type(labels)=<class 'numpy.ndarray'>
Shape of (4,)
Shape of (4,)
Type of type(predictions)=<class 'numpy.ndarray'>
Type of type(labels)=<class 'numpy.ndarray'>
Shape of (4,)
Shape of (4,)


Step,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Micro,F1 Weighted,Precision Macro,Precision Micro,Precision Weighted,Recall Macro,Recall Micro,Recall Weighted
63,1.097500,0.475202,0.841402,0.866019,0.841402,0.843592,0.871212,0.841402,0.851553,0.865719,0.841402,0.841402
126,0.383900,0.432957,0.814691,0.818625,0.814691,0.812231,0.860568,0.814691,0.825216,0.791536,0.814691,0.814691
189,0.281800,0.308675,0.879800,0.896656,0.879800,0.879280,0.902773,0.879800,0.881063,0.892207,0.879800,0.879800
252,0.200400,0.277068,0.898164,0.912240,0.898164,0.897111,0.921658,0.898164,0.899484,0.905633,0.898164,0.898164
315,0.135200,0.308870,0.894825,0.910360,0.894825,0.894403,0.917921,0.894825,0.897033,0.905019,0.894825,0.894825
378,0.108200,0.332039,0.894825,0.910134,0.894825,0.894178,0.916722,0.894825,0.895615,0.905161,0.894825,0.894825
441,0.081900,0.373421,0.891486,0.907656,0.891486,0.891693,0.911070,0.891486,0.893316,0.905254,0.891486,0.891486


Type of type(predictions)=<class 'numpy.ndarray'>
Type of type(labels)=<class 'numpy.ndarray'>
Shape of (4,)
Shape of (4,)
Type of type(predictions)=<class 'numpy.ndarray'>
Type of type(labels)=<class 'numpy.ndarray'>
Shape of (4,)
Shape of (4,)


TrainOutput(global_step=441, training_loss=0.27899570827311126, metrics={'train_runtime': 3511.8255, 'train_samples_per_second': 14.238, 'train_steps_per_second': 0.178, 'total_flos': 843291718778880.0, 'train_loss': 0.27899570827311126, 'epoch': 17.64})

In [ ]:
trainer.evaluate()

Type of type(predictions)=<class 'numpy.ndarray'>
Type of type(labels)=<class 'numpy.ndarray'>
Shape of (4,)
Shape of (4,)


{'eval_loss': 0.27706798911094666,
 'eval_accuracy': 0.8981636060100167,
 'eval_f1_macro': 0.9122397640750749,
 'eval_f1_micro': 0.8981636060100167,
 'eval_f1_weighted': 0.8971114223634937,
 'eval_precision_macro': 0.9216578612895938,
 'eval_precision_micro': 0.8981636060100167,
 'eval_precision_weighted': 0.8994842983190149,
 'eval_recall_macro': 0.9056329585751128,
 'eval_recall_micro': 0.8981636060100167,
 'eval_recall_weighted': 0.8981636060100167,
 'eval_runtime': 22.2396,
 'eval_samples_per_second': 26.934,
 'eval_steps_per_second': 1.709,
 'epoch': 17.64}

In [ ]:
trainer.evaluate(tokenized_dataset["test"], metric_key_prefix="test")

In [ ]:
predictions = trainer.predict(tokenized_dataset["test"]).label_ids
print(predictions.shape)

In [ ]:
# Showing the sentence, label and predicted label
for i in range(10):
    print(tokenized_dataset["test"][i]["sentence"])
    print(tokenized_dataset["test"][i]["label"])
    print(id2label[predictions[i]])

In [ ]:
wandb.finish()

eval/accuracy,▃▁▆███▇█
eval/f1_macro,▅▁▇█████
eval/f1_micro,▃▁▆███▇█
eval/f1_weighted,▄▁▇█████
eval/loss,█▇▂▁▂▃▄▁
eval/precision_macro,▂▁▆██▇▇█
eval/precision_micro,▃▁▆███▇█
eval/precision_weighted,▃▁▆███▇█
eval/recall_macro,▆▁▇█████
eval/recall_micro,▃▁▆███▇█
eval/recall_weighted,▃▁▆███▇█
